In [1]:
import pandas as pd
import numpy as np
import dash
import flask
from dash.dependencies import Input, Output
import dash_core_components as dcc
import dash_html_components as html
from pandas_datareader import data as web
import datetime as dt
import pymongo
import datetime

In [2]:
from dotenv import load_dotenv
load_dotenv()
import os
password = os.getenv("PASSWORD")
username = os.getenv("USERNAME")

In [8]:
def generate_object():
    # Connect to mongo using the credentials from .env file
    client = pymongo.MongoClient('mongodb://%s:%s@127.0.0.1' % (username, password))
    # Select the location based on the wordcount (1grams, 2grams, 3grams, etc.), by counting spaces
    db = client[str(query.count(' ')+1)+'grams']
    output=dict()
    output['word']=query
    # Build df
    df = pd.DataFrame(list(db[language].find({ "word": query})));
    df=df.dropna(how='all')
    df = df.sort_values(by=['time'])
    df['year'] = [date.year for date in df['time']]
    df['day'] = [date.timetuple().tm_yday for date in df['time']]
    # Pull out beginning part of date (YY-MM-DD)
    df['time'] = [str(t)[:10] for t in df['time']]
    # Convert date to date object
    #print("Before converting to date object, df.head(10) = ")
    #print(df.head(10))
    df['time'] = [dt.datetime.strptime(t, '%Y-%m-%d').date() for t in df['time']]
    # Remove dates before 2010
    #print("Before removing early dates, shape is", df.shape)
    df=df[df['time']>=(dt.date(2009,8,1))]
    #print("After removing early dates, shape is", df.shape)
    # Calculate min, max, and median rank
    output['maxrank']=int(min(df['rank'].values))
    output['minrank']=int(max(df['rank'].values))
    output['medianrank']=int(np.round(np.median(df['rank'].values)))
    # Index df by date
    #print("setting time as index...")
    df.set_index('time',inplace=True)
    #print("set time as index")
    # Drop the id field (used for indexing in the database)
    df.drop(columns=["_id"]);
    # Sort days
    #print("sorting by date...")
    df.sort_values(by='time',ascending=True,inplace=True)
    #print("sorted by date")
    # Convert time back to a string
    df.index=[t.strftime("%Y-%m-%d") for t in df.index]
    # Find the date on which the rank reached its first and most recent (could be the same day) minimum and maximum 
    #output['firstmax']=df[df['rank']==output['maxrank']].index[-1]
    #output['lastmax']=df[df['rank']==output['maxrank']].index[0]
    #output['firstmin']=df[df['rank']==output['minrank']].index[-1]
    #output['lastmin']=df[df['rank']==output['minrank']].index[0]
    # Send dates and metrics as arrays to the output dict
    output['dates']=df.index.values.tolist()
    if 'ranks' in metric:
        output['ranks']=[int(r) for r in df['rank'].values] # Convert from int64 to Python integers
    if 'counts' in metric:
        output['counts']=[int(c) for c in df['counts'].values] # Convert from int64 to Python integers
    if 'frequencies' in metric:
        output['frequencies']=[float(f) for f in df['freq'].values] # Convert from float64 Python float
    return output

In [11]:
query='halloween'
language='en'
metric = ['ranks','counts','frequencies']

In [12]:
generate_object()

{'word': 'halloween',
 'maxrank': 80,
 'minrank': 77483,
 'medianrank': 27055,
 'dates': ['2009-08-01',
  '2009-08-02',
  '2009-08-06',
  '2009-08-07',
  '2009-08-08',
  '2009-08-09',
  '2009-08-10',
  '2009-08-11',
  '2009-08-12',
  '2009-08-13',
  '2009-08-14',
  '2009-08-15',
  '2009-08-16',
  '2009-08-17',
  '2009-08-18',
  '2009-08-19',
  '2009-08-20',
  '2009-08-21',
  '2009-08-22',
  '2009-08-23',
  '2009-08-24',
  '2009-08-25',
  '2009-08-26',
  '2009-08-27',
  '2009-08-28',
  '2009-08-29',
  '2009-08-30',
  '2009-08-31',
  '2009-09-01',
  '2009-09-02',
  '2009-09-03',
  '2009-09-04',
  '2009-09-05',
  '2009-09-06',
  '2009-09-07',
  '2009-09-08',
  '2009-09-09',
  '2009-09-10',
  '2009-09-11',
  '2009-09-12',
  '2009-09-13',
  '2009-09-14',
  '2009-09-15',
  '2009-09-16',
  '2009-09-17',
  '2009-09-18',
  '2009-09-19',
  '2009-09-20',
  '2009-09-21',
  '2009-09-22',
  '2009-09-23',
  '2009-09-24',
  '2009-09-25',
  '2009-09-26',
  '2009-09-27',
  '2009-09-28',
  '2009-09-29',


In [ ]:
#!flask/bin/python
from flask import Flask
from flask import abort

app = Flask(__name__)

@app.route('/api/<str:query>', methods=['GET'])
def get_data(query):
    if len(query) == 0:
        abort(404)
    # Pull the language from the URL params, e.g. 'en', 'es', 'ru'
    #language = str(request.args.get('language'))
    # For now, we're just using english
    if language is None:
        language = 'en'
    # Pull the metric from the URL params, e.g. 'ranks','counts','frequencies'
    metric = str(request.args.get('metric'))
    if metric is None:
        metric = ['ranks','counts','frequencies']
    return jsonify(generate_object())

if __name__ == '__main__':
    app.run(debug=True, port='3001')